This notebook will be used for the Capstone Project from the Coursera Course <b>Applied Data Science Capstone</b>

In [1]:
import numpy as np
import pandas as pd
import requests

!conda install -c conda-forge geopy --yes # uncomment this line when not already installed
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line when not already installed
import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.1
  latest version: 4.9.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.11.8  |       ha878542_0         145 KB  conda-forge
    certifi-2020.11.8          |   py36h5fab9bb_0         150 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         392 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch

In [8]:
# read the statistische Stadtteile from the web
r = requests.get('https://de.wikipedia.org/wiki/Statistische_Gliederung_von_Nürnberg')
c = r.content
dfs = pd.read_html(c)

In [13]:
dfs [1][85:90] # Attention: this table needs to be reformatted because it has in-table headers

,Nr.,Name des statistischen Bezirks/Stadtteils,Fläche (ha)[Anm. 1],Einwohner[Anm. 2],Quelle
,0,Statistischer Stadtteil 0Altstadt und engere Innenstadt,"597,99",58826,[16]
85,86,Buchenbühl,3140,2257,[18]
86,87,Flughafen,3659,-,[18]
87,9,Statistischer Stadtteil 9Östliche Außenstadt,212955,46517,[16]
88,90,St. Jobst,1832,9574,[18]
89,91,Erlenstegen,3436,4009,[18]


# Get neighborhood data for Nuremberg

In [2]:
#49.470488°, 11.040573°

d7= {"Neighborhood": ["Westfriedhof","Schniegling","Wetzendorf","Buch","Thon","Almoshof","Kraftshof","Neunhof","Boxdorf","Großgründlach"],
    "Latitude" : [49.46343, 49.469519, 49.470488, 49.496803, 49.478333, 49.491389, 49.5125, 49.519444, 49.516389, 49.527222],
    "Longitude" : [11.04547, 11.028364, 11.040573, 11.045414, 11.058333, 11.061389, 11.046667, 11.05, 11.024722, 11.008333]
    }

d8= {"Neighborhood": ["Schleifweg","Schoppershof","Schafhof","Marienberg","Ziegelstein","Mooshof","Buchenbühl"],
    "Latitude" : [49.4714,49.469444, 49.480095,49.47976,49.49,49.488335,49.504167 ],
    "Longitude" : [11.0912,11.109722, 11.11832,11.090212,11.1036111,11.094388,11.11]
     }
# Latitude of Marienberg manually corrected, was nearly the same as for Schleifweg on Wikipedia
# here I hope to get data from the statistics bureau in Nbg

In [3]:
city_data = pd.DataFrame(data= d7)
city_data = city_data.append(pd.DataFrame(data= d8)).reset_index(drop=True)
city_data

,Neighborhood,Latitude,Longitude
0,Westfriedhof,49.463430,11.045470
1,Schniegling,49.469519,11.028364
2,Wetzendorf,49.470488,11.040573
3,Buch,49.496803,11.045414
4,Thon,49.478333,11.058333
5,Almoshof,49.491389,11.061389
6,Kraftshof,49.512500,11.046667
7,Neunhof,49.519444,11.050000
8,Boxdorf,49.516389,11.024722
9,Großgründlach,49.527222,11.008333


In [2]:
address = 'Nürnberg, DE'

geolocator = Nominatim(user_agent="de_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))

lat_nbg = location.latitude
lng_nbg = location.longitude

The geograpical coordinates of Nürnberg, DE are 49.453872, 11.077298.


In [6]:
# visualize the location of the neighborhoods within Nürnberg

map_nbg = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(city_data['Latitude'], city_data['Longitude'], city_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nbg)  
    
map_nbg

# Configure foursquare

In [3]:
CLIENT_ID = '1ASA1R5SJNI4KKZDE0VT02FRVCZPTLGRT35LWIICH2VGSSYW' # your Foursquare ID
CLIENT_SECRET = 'AQEUCQQX3WF40RDYNTMNFCVRNNV5OY1CC3MCVOSD212UVEKL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1ASA1R5SJNI4KKZDE0VT02FRVCZPTLGRT35LWIICH2VGSSYW
CLIENT_SECRET:AQEUCQQX3WF40RDYNTMNFCVRNNV5OY1CC3MCVOSD212UVEKL


# Get the gyms in Nuremberg - via neighborhood
Approach: search for all venues and filter them for gyms 

In [8]:
# function from the Labs extended to return also venue id
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'],
            v['venue']['id']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category',
                  'Venue Id']
    
    return(nearby_venues)

In [89]:
label = city_data.Neighborhood
lat = city_data.Latitude
lng = city_data.Longitude
nbg_venues = getNearbyVenues(label, lat, lng, radius=700)
nbg_venues

Westfriedhof
Schniegling
Wetzendorf
Buch
Thon
Almoshof
Kraftshof
Neunhof
Boxdorf
Großgründlach
Schleifweg
Schoppershof
Schafhof
Marienberg
Ziegelstein
Mooshof
Buchenbühl


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Id
0,Westfriedhof,49.463430,11.045470,Pegnitztal West,49.460914,11.044865,Park,4c27a1a3c11dc9b659eb2a24
1,Westfriedhof,49.463430,11.045470,Anderland,49.461055,11.053115,Pizza Place,4be0723c2aed2d7f5065c86f
2,Westfriedhof,49.463430,11.045470,Lidl,49.457695,11.042520,Supermarket,4c38afcedfb0e21eb604afa8
3,Westfriedhof,49.463430,11.045470,H Bienweg,49.464715,11.047678,Bus Stop,4f0c5aece4b0065899dd8f9b
4,Westfriedhof,49.463430,11.045470,H Westfriedhof,49.461614,11.049716,Tram Station,4d20847f756e8cfa8bfe6c54
...,...,...,...,...,...,...,...,...
123,Mooshof,49.488335,11.094388,NORMA,49.487058,11.103422,Supermarket,4e8057e229c2827511a140ce
124,Buchenbühl,49.504167,11.110000,Bayerischer Staatsforst Buchenbühl,49.503069,11.108346,Forest,522071a911d2293cbab0aff5
125,Buchenbühl,49.504167,11.110000,Lotto Bayern,49.502020,11.110880,Lottery Retailer,5b8414f9dd70c5002cc25a38
126,Buchenbühl,49.504167,11.110000,KFZ Sachverständigenbüro Kramm,49.499630,11.111780,Business Service,56e2a7c8498e13426959454f


In [105]:
#nbg_venues["Venue Category"].unique()

In [91]:
# check which values are duplicates, i.e. were returned for more than one neighborhood
nbg_venues["lat-long"] = nbg_venues["Venue Latitude"].astype(str) + "-" + nbg_venues["Venue Longitude"].astype(str)
nbg_venues[nbg_venues.duplicated(subset=["lat-long"])]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Id,lat-long
20,Wetzendorf,49.470488,11.040573,H Krematorium,49.466328,11.040924,Bus Stop,4f0c5d87e4b0f184bb737066,49.46632829967994-11.0409239651026
25,Buch,49.496803,11.045414,Christoph Waldner,49.501332,11.045126,Construction & Landscaping,5d2e6f8d531dc50008d2dc16,49.501332-11.045126
106,Marienberg,49.479760,11.090212,Gaststätte Marienberg,49.477480,11.091650,Italian Restaurant,4e189773b61c8f9ada4d259f,49.47747980268108-11.091649636034846
119,Mooshof,49.488335,11.094388,Tucherhof,49.485101,11.090337,German Restaurant,4c5e9cd894fd0f4745a7ce45,49.485101491142586-11.090336521769148
121,Mooshof,49.488335,11.094388,Sport Park Nord,49.488872,11.097466,Gym / Fitness Center,4cb5e85b64998cfa706c11a2,49.48887228309483-11.097466448722226
122,Mooshof,49.488335,11.094388,Futterstubn,49.489692,11.103054,Burger Joint,51b1b7fa498ec1af7bb7d8c9,49.489692-11.103054
123,Mooshof,49.488335,11.094388,NORMA,49.487058,11.103422,Supermarket,4e8057e229c2827511a140ce,49.48705756508181-11.103421942701248


In [93]:
nbg_gyms = nbg_venues[nbg_venues["Venue Category"].str.lower().str.contains("gym")]
# drop duplicates and reset the index
nbg_gyms = nbg_gyms.drop_duplicates(subset=["lat-long"]).reset_index()
nbg_gyms

,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Id,lat-long
0,92,Schafhof,49.480095,11.118320,Café Kraft,49.479874,11.122704,Climbing Gym,4e5f9e30ae600b89d7c62f65,49.479874-11.122704
1,114,Ziegelstein,49.490000,11.103611,Sport Park Nord,49.488872,11.097466,Gym / Fitness Center,4cb5e85b64998cfa706c11a2,49.48887228309483-11.097466448722226


Issues:
if radius is too less then only few venues are found in each neighborhood
if radius is too big, there is too much overlapping in the venues found


In [19]:
# visualize the gyms on the map
# create map
map_nbg = folium.Map(location=[latitude, longitude], zoom_start=12)

# add the gyms as blue circle markers
for lat, lng, label in zip(nbg_gyms["Venue Latitude"], nbg_gyms["Venue Longitude"], nbg_gyms["Venue"]):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(map_nbg)

#display map    
map_nbg

# Get the gyms in Nuremberg - via direct search from center coordinates
Approach: search for all "Gym/Fitness Center" within a radius from city center 

In [54]:
def getNearbyCategory(name, latitude, longitude, categorylist, radius):

# using the foursquare API...
# function to get the venues from a categorylist (can also contain only one value) that are within
# a certain radius of a point definded by name, latitude and longitude
# name is only used for having it in the returned dataframe. it is not needed for the foursquare quiery

    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, categorylist, radius, LIMIT)

    results = requests.get(url).json()["response"]['venues']

    venues_list = []
    venues_list.append([(
            name, 
            latitude, 
            longitude,
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],  
            v['categories'][0]['name'],
            v['id']
            ) for v in results])
    venues_list= venues_list[0]
    nearby_venues = pd.DataFrame(data=venues_list)

    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category',
                  'Venue Id']
    return(nearby_venues)

In [55]:
# instead of using "query" which matches venue name, use "CategoryId" which matches for a (list of) categories!

name = "Nürnberg"
lat = lat_nbg # 49.453872
lng = lng_nbg # 11.077298

search_category = '4bf58dd8d48988d175941735' # category ID for "Gym/Fitness Center"
radius = 5000  

nbg_gyms = getNearbyCategory(name, lat, lng, search_category, radius)   
nbg_gyms

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Id
0,Nürnberg,49.453872,11.077298,FIT ONE,49.449467,11.074092,Gym / Fitness Center,586ca64e18dc536fecf25667
1,Nürnberg,49.453872,11.077298,FIT STAR Fitnessstudio Nürnberg-Zentrum,49.448212,11.083224,Gym / Fitness Center,577d6a44498eadda3146ab2d
2,Nürnberg,49.453872,11.077298,Schwimmbad Ebensee,49.467580,11.139481,Gym Pool,52dd6b8c498e35e77df9f275
3,Nürnberg,49.453872,11.077298,FIT STAR Fitnessstudio Fürth-Südstadt,49.459940,11.006402,Gym / Fitness Center,577d68b0498eea8e9db20e71
4,Nürnberg,49.453872,11.077298,clever fit Fitnessstudio Nürnberg,49.423984,11.068681,Gym,50a2da52f2e7ce3e7a58855e
5,Nürnberg,49.453872,11.077298,Kieser Training,49.444779,11.109701,Gym / Fitness Center,5310437c11d2a01b7c45b5b6
6,Nürnberg,49.453872,11.077298,Fitness First Lifestyle Club,49.450252,11.070769,Gym / Fitness Center,4bd9a7ece914a593034557fa
7,Nürnberg,49.453872,11.077298,Post SV Fitlounge,49.453875,11.089603,Gym / Fitness Center,4b7d849af964a520e1c32fe3
8,Nürnberg,49.453872,11.077298,BodytecPoint,49.471080,11.022999,Gym / Fitness Center,52652161498efba58d4440e0
9,Nürnberg,49.453872,11.077298,Powerbox Bei Eugen,49.455033,11.037993,Gym / Fitness Center,5a7593a4e179101a278e3371


In [6]:
# visualize the gyms on the map
# create map
map_nbg = folium.Map(location=[lat_nbg, lng_nbg], zoom_start=12)

# add the gyms as blue circle markers
for lat, lng, label in zip(nbg_gyms["Venue Latitude"], nbg_gyms["Venue Longitude"], nbg_gyms["Venue"]):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(map_nbg)

#display map    
map_nbg

In [45]:
# using different radius seems to lead to not only more/less but DIFFERENT results. Check this

name = "Nürnberg"
lat = 49.453872
lng = 11.077298

search_category = '4bf58dd8d48988d175941735' # category ID for "Gym/Fitness Center"
radius = [500, 1000, 2000, 3000, 4000, 5000] 

df_ids = pd.DataFrame()
for rad in radius: 
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, search_category, rad, LIMIT)
    
    results = requests.get(url).json()["response"]['venues']

    venues_list = []
    venues_list.append([(
            v['name']#, v['id']
            ) for v in results])
    venues_list= venues_list[0]
    #print(len(venues_list))
    padding = (50-len(venues_list)) * [0]  # attention: 50 hardcoded means the maximum amount of venues returned for a radius must not be bigger than 50 
    venues_list = venues_list + padding
    column = "radius"+str(rad)
    df_ids[column] = venues_list

df_ids 

,radius500,radius1000,radius2000,radius3000,radius4000,radius5000
0,FIT ONE,FIT ONE,FIT ONE,FIT ONE,FIT ONE,FIT ONE
1,Peter Althof Boxgym,FIT STAR Fitnessstudio Nürnberg-Zentrum,FIT STAR Fitnessstudio Nürnberg-Zentrum,FIT STAR Fitnessstudio Nürnberg-Zentrum,FIT STAR Fitnessstudio Nürnberg-Zentrum,FIT STAR Fitnessstudio Nürnberg-Zentrum
2,"Zumba-Studio ""Move It""",Fitness First Lifestyle Club,Jumpers Fitness,Jumpers Fitness,Jumpers Fitness,Schwimmbad Ebensee
3,Zumbafit,Post SV Fitlounge,Kieser Training,Kieser Training,clever fit Fitnessstudio Nürnberg,FIT STAR Fitnessstudio Fürth-Südstadt
4,Bodyvision,DAS STUDIO – Aerobic in der City,Fitness First Lifestyle Club,Kieser Training,Post SV Fitlounge,clever fit Fitnessstudio Nürnberg
5,0,Novotel Gym & Spa,Post SV Fitlounge,Post SV Fitlounge,Kieser Training,Kieser Training
6,0,Figura,Lady Style,Fitness First Lifestyle Club,Kieser Training,BodytecPoint
7,0,Le Meridien Gym & Sauna,DAS STUDIO – Aerobic in der City,Powerbox Bei Eugen,BodytecPoint,Fitness First Lifestyle Club
8,0,Bodystreet Nürnberg Zentrum,Sporthalle des Berufsbildungszentrums,DAS STUDIO – Aerobic in der City,Fitness First Lifestyle Club,Post SV Fitlounge
9,0,Fitness & Wellness NH Hotel,Sibeliusbad,Lady Style,Powerbox Bei Eugen,Powerbox Bei Eugen


In [10]:
name = "Nürnberg"
lat = 49.453872
lng = 11.077298

search_category = '4bf58dd8d48988d175941735' # category ID for "Gym/Fitness Center" 

df500 = getNearbyCategory(name, lat, lng, search_category, 500).sort_values(by="Venue Id")
df1000 = getNearbyCategory(name, lat, lng, search_category, 1000).sort_values(by="Venue Id")
df2000 = getNearbyCategory(name, lat, lng, search_category, 2000).sort_values(by="Venue Id")
df3000 = getNearbyCategory(name, lat, lng, search_category, 3000).sort_values(by="Venue Id")
df4000 = getNearbyCategory(name, lat, lng, search_category, 4000).sort_values(by="Venue Id")
df5000 = getNearbyCategory(name, lat, lng, search_category, 5000).sort_values(by="Venue Id")

In [83]:
# continue here, goal is to replace the df_ids with info extracted from the df500... and so on 
# in order to have less foursquare queries
df500.head()
radius = [500, 1000]#, 2000, 3000, 4000, 5000]
df_list = ["df500", "df1000"]
for rad,df in radius,df_list:
    print(rad,df)
    df_rad = pd.DataFrame()
    column = "radius"+str(rad)
    venues_list = df["Venue"].to_list()
    #padding = (50-len(venues_list)) * [0]  # attention: 50 hardcoded means the maximum amount of venues returned for a radius must not be bigger than 50 
    #venues_list = venues_list + padding  
    #df_rad[column] = venues_list
df_rad

500 1000


TypeError: 'int' object is not subscriptable

In [11]:
df500venues = df500["Venue Id"].to_list() 
df1000venues = df1000["Venue Id"].to_list() 
df2000venues = df2000["Venue Id"].to_list() 
df3000venues = df3000["Venue Id"].to_list() 
df4000venues = df4000["Venue Id"].to_list() 
df5000venues = df5000["Venue Id"].to_list() 

In [12]:
# I assume that every venue that is within a radius of 500 is also within a radius of 1000. Let's check if this is true
print("Are all venues from radius=500 also in radius=1000? ", set(df500venues).intersection(df1000venues) == set(df500venues))
# I assume that every venue that is within a radius of 500 is also within a radius of 2000. Let's check if this is true
print("Are all venues from radius=500 also in radius=2000? ",set(df500venues).intersection(df2000venues) == set(df500venues))
print("Number of venues from radius=500 that are also in radius=2000: ", len(set(df500venues).intersection(df2000venues)))
# only 2 venues from the 500 radius are also in the 2000 radius, isnt this strange? let's check for 1000 vs 2000
print("Are all venues from radius=1000 also in radius=2000? ",set(df1000venues).intersection(df2000venues) == set(df1000venues))
print("Number of venues from radius=1000 that are also in radius=2000: ", len(set(df1000venues).intersection(df2000venues)))

Are all venues from radius=500 also in radius=1000?  True
Are all venues from radius=500 also in radius=2000?  False
Number of venues from radius=500 that are also in radius=2000:  2
Are all venues from radius=1000 also in radius=2000?  False
Number of venues from radius=1000 that are also in radius=2000:  14


So I can conclude that depending on the radius given in the search query Foursquare returns different venues. I assumed that it would just return more venues with an increasing radius. But this is not the case. 

# Get the ratings of the gyms

In [57]:
# for quickly checking the forsquare response manually... if nothing is returned check result variable for 'quota exceeded'
venue_id = nbg_gyms.loc[8,'Venue Id']
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
#result['response']['venue']
try: 
            data_available = result['response']['venue']
except:
            print(result['meta']['errorDetail'])
#print(result['meta']['errorDetail'])

Quota exceeded


In [19]:
def getRatingData(venue_ids):
# for a given venue_id list, this fuction extracts, for each venue in the list, the following data from Foursquare:
# Rating, Number of Tips, Number of Likes, Number of Dislikes
# if no rating data is available Nan is written
# a Dataframe with the rating info is returned
# if no data is returned from Foursquare, this function returns the error message 
    
    rating_info=[]
    for v_id in venue_ids:
        print(v_id)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(
            v_id, 
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
            
        # make the GET request
        result = requests.get(url).json()
       
        # check if data is there, if not return with error message
        try: 
            data_available = result['response']['venue']
        
        except:
            error_msg = result['meta']['errorDetail']
            return(error_msg)
        
        # extract needed info from data
        else:
            try:
                rating = result['response']['venue']['rating']
            except:
                rating = np.nan
            
            # get number of tips
            nr_tips = result['response']['venue']['tips']['count']
       
            try:
                nr_likes = result['response']['venue']['likes']['count']
            except:
                nr_likes = np.nan
            
            # get number of dislikes
            try:
                nr_dislikes = result['response']['venue']['dislike']['count']
            except:
                nr_dislikes = np.nan
    
            rating_info.append([
                v_id,
                rating, 
                nr_tips, 
                nr_likes,
                nr_dislikes]) 

            rating_data = pd.DataFrame(data=rating_info)
            rating_data.columns = ['Venue Id', 'Rating', "No of Tips", "No of Likes", "No of Dislikes"] 
              
    return(rating_data)
           

In [20]:
gym_ratings = getRatingData(nbg_gyms["Venue Id"])

586ca64e18dc536fecf25667
7.8
577d6a44498eadda3146ab2d
nan
52dd6b8c498e35e77df9f275
nan
577d68b0498eea8e9db20e71
nan
52652161498efba58d4440e0
nan
50a2da52f2e7ce3e7a58855e
7.3
5310437c11d2a01b7c45b5b6
nan
4bd9a7ece914a593034557fa
8.6
4b7d849af964a520e1c32fe3
nan
5a7593a4e179101a278e3371
nan
4ea3ca007ee52a35dd77700a
nan
4e81eb795503ca7789259b2e
nan
4cb5e85b64998cfa706c11a2
nan
51670c17e4b02703e1e4a095
8.1
4b99429df964a520676e35e3
6.7
51d5973b498e8230f9a7f24b
nan
580cb4ed38fa79522b4ce4e2
nan
542e69d4498efe79608418b0
nan
50d36db8e4b0abb8c47f790f
nan
523348a611d2d7333803e886
nan
4e823e8d4901000af3a71620
nan
50705bf6e4b0d7b67eed8a0a
nan
5837521eba883710b2c85ecc
nan
58c6ff90d25ded210443be22
nan
524d0e53498e841847909a16
nan
4b7980cef964a52036fd2ee3
7.7
4fa10232e4b07aaa44dee796
8.7
4f02d449c2ee2cc871333ab5
8.4
4e5f9e30ae600b89d7c62f65
8.8
4cc74269be87468842747daf
nan
4eba6334dab495868c4d79ab
7.6
4b71aa3af964a5209d542de3
8.3
4e336d6d52b11c7462230567
nan
4e560919c65be3cea46095cd
nan
50b21e79e4b0ed

In [66]:
gym_ratings[gym_ratings["No of Likes"]>0]

,Venue Id,Rating,No of Tips,No of Likes,No of Dislikes,Cluster
0,586ca64e18dc536fecf25667,7.8,1,8,-1.0,3
1,577d6a44498eadda3146ab2d,-1.0,0,1,-1.0,0
2,52dd6b8c498e35e77df9f275,-1.0,0,1,-1.0,0
3,577d68b0498eea8e9db20e71,-1.0,0,1,-1.0,0
4,52652161498efba58d4440e0,-1.0,0,1,-1.0,0
5,50a2da52f2e7ce3e7a58855e,7.3,2,6,-1.0,3
6,5310437c11d2a01b7c45b5b6,-1.0,0,1,-1.0,0
7,4bd9a7ece914a593034557fa,8.6,3,34,-1.0,1
8,4b7d849af964a520e1c32fe3,-1.0,2,5,-1.0,0
11,4e81eb795503ca7789259b2e,-1.0,0,3,-1.0,0


# Cluster gyms according to their rating data

In [23]:
# replace Nan with  -1 as Kmeans can't handle Nans
gym_ratings = gym_ratings.fillna(-1)

,Venue Id,Rating,No of Tips,No of Likes,No of Dislikes
0,586ca64e18dc536fecf25667,7.8,1,8,-1.0
1,577d6a44498eadda3146ab2d,-1.0,0,1,-1.0
2,52dd6b8c498e35e77df9f275,-1.0,0,1,-1.0
3,577d68b0498eea8e9db20e71,-1.0,0,1,-1.0
4,52652161498efba58d4440e0,-1.0,0,1,-1.0
5,50a2da52f2e7ce3e7a58855e,7.3,2,6,-1.0
6,5310437c11d2a01b7c45b5b6,-1.0,0,1,-1.0
7,4bd9a7ece914a593034557fa,8.6,3,34,-1.0
8,4b7d849af964a520e1c32fe3,-1.0,2,5,-1.0
9,5a7593a4e179101a278e3371,-1.0,0,0,-1.0


In [31]:
# set number of clusters
kclusters = 4

gym_ratings_clustering = gym_ratings.drop(['Venue Id','No of Dislikes'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0)
kmeans.fit(gym_ratings_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:30] 

array([3, 0, 0, 0, 0, 3, 0, 1, 0, 0, 0, 0, 0, 2, 3, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 2, 1, 1, 0], dtype=int32)

In [32]:
gym_ratings["Cluster"] = kmeans.labels_

Cluster 0: "No rating available"

In [67]:
print((gym_ratings[gym_ratings["Cluster"] == 0]).shape)
gym_ratings[gym_ratings["Cluster"] == 0]

(32, 6)


,Venue Id,Rating,No of Tips,No of Likes,No of Dislikes,Cluster
1,577d6a44498eadda3146ab2d,-1.0,0,1,-1.0,0
2,52dd6b8c498e35e77df9f275,-1.0,0,1,-1.0,0
3,577d68b0498eea8e9db20e71,-1.0,0,1,-1.0,0
4,52652161498efba58d4440e0,-1.0,0,1,-1.0,0
6,5310437c11d2a01b7c45b5b6,-1.0,0,1,-1.0,0
8,4b7d849af964a520e1c32fe3,-1.0,2,5,-1.0,0
9,5a7593a4e179101a278e3371,-1.0,0,0,-1.0,0
10,4ea3ca007ee52a35dd77700a,-1.0,0,0,-1.0,0
11,4e81eb795503ca7789259b2e,-1.0,0,3,-1.0,0
12,4cb5e85b64998cfa706c11a2,-1.0,0,4,-1.0,0


Cluster 1: "high rating and high number of likes"

In [33]:
gym_ratings[gym_ratings["Cluster"] == 1]

,Venue Id,Rating,No of Tips,No of Likes,No of Dislikes,Cluster
7,4bd9a7ece914a593034557fa,8.6,3,34,-1.0,1
27,4f02d449c2ee2cc871333ab5,8.4,8,51,-1.0,1
28,4e5f9e30ae600b89d7c62f65,8.8,7,42,-1.0,1


Cluster 2: "medium number of likes"

In [34]:
gym_ratings[gym_ratings["Cluster"] == 2]

,Venue Id,Rating,No of Tips,No of Likes,No of Dislikes,Cluster
13,51670c17e4b02703e1e4a095,8.1,6,15,-1.0,2
25,4b7980cef964a52036fd2ee3,7.7,3,20,-1.0,2
26,4fa10232e4b07aaa44dee796,8.7,3,20,-1.0,2
30,4eba6334dab495868c4d79ab,7.6,2,13,-1.0,2
31,4b71aa3af964a5209d542de3,8.3,0,18,-1.0,2


Cluster 3: "medium rating, low number of likes"

In [35]:
gym_ratings[gym_ratings["Cluster"] == 3]

,Venue Id,Rating,No of Tips,No of Likes,No of Dislikes,Cluster
0,586ca64e18dc536fecf25667,7.8,1,8,-1.0,3
5,50a2da52f2e7ce3e7a58855e,7.3,2,6,-1.0,3
14,4b99429df964a520676e35e3,6.7,1,8,-1.0,3


In [70]:
# merge rating and cluster info back in nbg_gyms dataframe
nbg_gyms = pd.merge(nbg_gyms, gym_ratings, how='left', on='Venue Id')

In [76]:
#create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
rainbow = ['#8000ff', '#2adddd', '#006400', '#ff0000'] # manually overwritten to make cluster 3 visible on the map, olive green was hard to see

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nbg_gyms['Venue Latitude'], nbg_gyms['Venue Longitude'], nbg_gyms['Venue'], nbg_gyms['Cluster']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [49]:
nbg_gyms = nbg_gyms.drop(['Rating_x', 'No of Tips_x', 'No of Likes_x',
       'No of Dislikes_x', 'Cluster_x', 'Rating_y', 'No of Tips_y',
       'No of Likes_y', 'No of Dislikes_y', 'Cluster_y'], axis=1)

In [73]:
rainbow

['#8000ff', '#2adddd', '#d4dd80', '#ff0000']

In [74]:
rainbow = ['#8000ff', '#2adddd', '#4ddd80', '#ff0000']

array([[5.00000000e-01, 0.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [1.66666667e-01, 8.66025404e-01, 8.66025404e-01, 1.00000000e+00],
       [8.33333333e-01, 8.66025404e-01, 5.00000000e-01, 1.00000000e+00],
       [1.00000000e+00, 1.22464680e-16, 6.12323400e-17, 1.00000000e+00]])